In [ ]:
import numpy as np
import pandas as pd
from math import isnan
import re
import matplotlib.pyplot as plt
from ape_producer.geom_reader import create_geom_df
from ape_producer.parse_report import make_cov_objects
from ape_producer.sign_conventions import signConventions, square_coordinates
from scipy.optimize import curve_fit
#import mplhep as hep
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 500)

In [ ]:
def select_sectors(df, wheel, station):
    return df[(df.wheel==wheel) & (df.station==station)]

In [ ]:
xmlFileAlign_new_2 = 'data/SM_10_6_01_2.xml'
xmlFileAlign_new_4 = 'data/SM_10_6_01_4.xml'
xmlFileAlign_new_8 = 'data/SM_10_6_01_8.xml'
xmlFileAlign_new_16 = 'data/SM_10_6_01_16.xml'
xmlFileAlign_new_31 = 'data/SM_10_6_01_31.xml'
xmlFileAlign_new_62 = 'data/SM_10_6_01_62.xml'
xmlFileAlign_new_125 = 'data/SM_10_6_01_125.xml'
xmlFileAlign_new_250 = 'data/SM_10_6_01_250.xml'
xmlFileAlign_new_500 = 'data/SM_10_6_01_500.xml'

In [ ]:
RyanData = pd.DataFrame()
RyanData = pd.read_csv("data/nmuon_hesse_df.csv") 
RyanData= RyanData.astype(float)
RyanData = RyanData.apply(pd.to_numeric)

In [ ]:
aligned_geom_dt_2, aligned_geom_csc_2  = create_geom_df(xmlFileAlign_new_2)
aligned_geom_dt_4, aligned_geom_csc_4  = create_geom_df(xmlFileAlign_new_4)
aligned_geom_dt_8, aligned_geom_csc_8  = create_geom_df(xmlFileAlign_new_8)
aligned_geom_dt_16, aligned_geom_csc_16  = create_geom_df(xmlFileAlign_new_16)
aligned_geom_dt_31, aligned_geom_csc_31  = create_geom_df(xmlFileAlign_new_31)
aligned_geom_dt_62, aligned_geom_csc_62  = create_geom_df(xmlFileAlign_new_62)
aligned_geom_dt_125, aligned_geom_csc_125  = create_geom_df(xmlFileAlign_new_125)
aligned_geom_dt_250, aligned_geom_csc_250 = create_geom_df(xmlFileAlign_new_250)
aligned_geom_dt_500, aligned_geom_csc_500  = create_geom_df(xmlFileAlign_new_500)

In [ ]:
aligned_geom_dt_2['njobs'] = 2
aligned_geom_dt_4['njobs'] = 4
aligned_geom_dt_8['njobs'] = 8
aligned_geom_dt_16['njobs'] = 16
aligned_geom_dt_31['njobs'] = 31
aligned_geom_dt_62['njobs'] = 62
aligned_geom_dt_125['njobs'] = 125
aligned_geom_dt_250['njobs'] = 250
aligned_geom_dt_500['njobs'] = 500

In [ ]:
from scipy.optimize import curve_fit
from scipy.stats import rv_continuous
from scipy.stats import norm
import uncertainties 
def calc_likelihood(x, width):
    return np.product(norm.pdf(x,0,width))
def norm_fit(x, c, scale):
    return c*norm.pdf(x, loc=0, scale=scale)
def calc_uncertainty(mean, width, values):
    offsets = np.linspace(-width+1e-10,width,100)
    liklihoods = np.array(list(map(lambda y: calc_likelihood(values, width+y), offsets)))
    liklihoods = np.array(liklihoods)/np.sum(liklihoods)
    try:
        popt, pcov = curve_fit(norm_fit, offsets, liklihoods ,maxfev = int(1e4) )
    except:
        plt.plot(offsets, liklihoods)
        print(offsets, liklihoods, values)
    return popt[1]
def sigma_and_unc(key, *args):
    values = select_sectors(*args)[key]
    mean, width = norm.fit(values, floc=0)
    unc = calc_uncertainty(mean, width, values)
    return uncertainties.ufloat(width, unc)

In [ ]:
data_frames = {
    "2":aligned_geom_dt_2,
    "4":aligned_geom_dt_4,
    "8":aligned_geom_dt_8,
    "16":aligned_geom_dt_16,
    "31":aligned_geom_dt_31,
    "62":aligned_geom_dt_62,
    "125":aligned_geom_dt_125,
    "250":aligned_geom_dt_250,
    "500":aligned_geom_dt_500,
}

In [ ]:
result_list = []
for i in range(-2, 3):
    for j in range (1, 5):
        for count, tdf in data_frames.items():
            dim_dict = {}
            #for dim in ['x', 'y', 'z', 'phix', 'phiy', 'phiz']:
            for dim in ['x']:
                if dim=='x':
                    ufloat = sigma_and_unc(dim, tdf, i, j)
                    dim_dict["{} std".format(dim)] =  ufloat.nominal_value
                    dim_dict["{} std unc".format(dim)] =  ufloat.std_dev
                elif j<4 and dim!='z':
                    ufloat = sigma_and_unc(dim, tdf, i, j)
                    dim_dict["{} std".format(dim)] =  ufloat.nominal_value
                    dim_dict["{} std unc".format(dim)] =  ufloat.std_dev
                else: 
                    dim_dict["{} std".format(dim)] =  0
                    dim_dict["{} std unc".format(dim)] =  0
            row_dict = {'returns:':count,'wheel':i, 'station':j, **dim_dict}
            result_list.append(row_dict)

In [ ]:
ReturnsDF=pd.DataFrame(result_list)
ReturnsDF = ReturnsDF.apply(pd.to_numeric)
ReturnsDF

In [ ]:
Result = pd.merge(RyanData, ReturnsDF)

In [ ]:
Result

In [ ]:
def DTPlot(wheel, station, option):
    fig, DTPlot = plt.subplots()
    
    DTPlot.plot(0, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 2 ) ][option].values, 'ro', label = '2 files')
    DTPlot.plot(1, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 4 ) ][option].values, 'ro', label = '4 files')
    DTPlot.plot(2, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 8 ) ][option].values, 'ro', label = '8 files')
    DTPlot.plot(3, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 16 ) ][option].values, 'ro', label = '16 files')
    DTPlot.plot(4, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 31 ) ][option].values, 'ro', label = '31 files')
    DTPlot.plot(5, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 62 ) ][option].values, 'ro', label = '62 files')
    DTPlot.plot(6, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 125 ) ][option].values, 'ro', label = '125 files')
    DTPlot.plot(7, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 250 ) ][option].values, 'ro', label = '250 files')
    DTPlot.plot(8, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 500) ][option].values, 'ro', label = '500 files')
    ax = plt.gca()
    ax.set_xticks([0,1,2,3,4,5,6,7,8] )
    ax.set_xticklabels(["2","4","8","16","31","62","125","250","500"])
    

In [ ]:
Result[(Result.wheel == 0) & (Result.station == 1) & (Result['returns:'] == 2)]['phiz std'].values

In [ ]:
#nMuons_mean
#nMuons_std
#hesse_unc_x
#hesse_unc_y
#hesse_unc_z
#hesse_unc_phix
#hesse_unc_phiy
#hesse_unc_phiz
#x std
#y std
#z std
#phix std
#phiy std
#phiz std
DTPlot(1,2, "x std")

In [ ]:
def DTPlot2opt(wheel, station, option1, option2):
    fig, DTPlot = plt.subplots()
    
    DTPlot.plot(0, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 2 ) ][option].values, 'ro', label = '2 files')
    DTPlot.plot(1, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 4 ) ][option].values, 'ro', label = '4 files')
    DTPlot.plot(2, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 8 ) ][option].values, 'ro', label = '8 files')
    DTPlot.plot(3, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 16 ) ][option].values, 'ro', label = '16 files')
    DTPlot.plot(4, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 31 ) ][option].values, 'ro', label = '31 files')
    DTPlot.plot(5, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 62 ) ][option].values, 'ro', label = '62 files')
    DTPlot.plot(6, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 125 ) ][option].values, 'ro', label = '125 files')
    DTPlot.plot(7, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 250 ) ][option].values, 'ro', label = '250 files')
    DTPlot.plot(8, Result[(Result.wheel == wheel) & (Result.station == station) & (Result['returns:'] == 500) ][option].values, 'ro', label = '500 files')
    ax = plt.gca()

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def sys_func(x, p0, p1):
    return (p0 ** 2 / x + p1 ** 2) ** 0.5

In [ ]:
Result

In [ ]:
def plot_wheel_station(ax, wheel, station, option):
    tempResults = Result[(Result.wheel==wheel) & (Result.station==station)]
    tempResults = tempResults[tempResults.nMuons_mean >50]
    if option == "x":
        option = "x std"
        option_error = tempResults.hesse_unc_x
    if option == "y":
        option = "y std"
        option_error = tempResults.hesse_unc_y
    if option == "z":
        option = "z std"
        option_error = tempResults.hesse_unc_z
    if option == "phix":
        option = "phix std"
        option_error = tempResults.hesse_unc_phix
    if option == "phiy":
        option = "phiy std"
        option_error = tempResults.hesse_unc_phiy
    if option == "phiz":
        option = "phiz std"
        option_error = tempResults.hesse_unc_phiz
    
    x = tempResults.nMuons_mean
    y = tempResults[option]
    popt, pcov = curve_fit(sys_func, x, y)
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = sys_func(x_fit, *popt)
    #plt.yscale("log")
    #plt.xscale("log")
    ax.plot(x_fit, y_fit)
    #ax.errorbar(x, y, yerr = tempResults.hesse_unc_x,linestyle='None')
    ax.scatter(x, y, label='wheel-station {}/{}: sys {:.2e}'.format(wheel, station, np.abs(popt[1]) ))
    ax.set_xlabel('n Muons')
    ax.set_ylabel(option)
    print("a: {} b: {}".format(*popt))
    

In [ ]:
def draw_wheel(wheel, option):
    fig, axs = plt.subplots(1,1, figsize=(10,5))
    plot_wheel_station(axs, wheel, 1, option)
    plot_wheel_station(axs, wheel, 2, option)
    plot_wheel_station(axs, wheel, 3, option)
    if option != "y": 
        plot_wheel_station(axs, wheel, 4, option)
    axs.set_title("Wheel {}".format(wheel))
    #hep.style.use(hep.style.ROOT)
    axs.legend()

In [ ]:
def DrawAllWheels(option):
    draw_wheel(-2, option)
    draw_wheel(-1, option)
    draw_wheel(0, option)
    draw_wheel(1, option)
    draw_wheel(2, option)

In [ ]:
Result[(Result.wheel==2) & (Result.station==4)]**2

In [ ]:
def plot_wheel_stationhesse(ax, wheel, station, option):
    tempResults = Result[(Result.wheel==wheel) & (Result.station==station)]
    tempResults = tempResults[tempResults.nMuons_mean >50]
    if option == "x":
        option = "x std"
        hess = "hesse_unc_x"
    if option == "y":
        option = "y std"
        hess = "hesse_unc_y"
    if option == "z":
        option = "z std"
        hess = "hesse_unc_z"
    if option == "phix":
        option = "phix std"
        hess = "hesse_unc_phix"
    if option == "phiy":
        option = "phiy std"
        hess = "hesse_unc_phiy"
    if option == "phiz":
        option = "phiz std"
        hess = "hesse_unc_phiz"
    
    x = tempResults.nMuons_mean
    y = (tempResults[option]**2-tempResults[hess]**2).abs()**.5
    #y = tempResults[hess]
    popt, pcov = curve_fit(sys_func, x, y)
    x_fit = np.linspace(min(x), max(x), 100)
    y_fit = sys_func(x_fit, *popt)
    #plt.yscale("log")
    #plt.xscale("log")
    ax.plot(x_fit, y_fit)
    #ax.errorbar(x, y, yerr = tempResults.hesse_unc_x,linestyle='None')
    ax.scatter(x, y, label='wheel-station {}/{}: sys {:.2e}'.format(wheel, station, np.abs(popt[1]) ))
    ax.set_xlabel('n Muons')
    ax.set_ylabel(option)
    print("a: {} b: {}".format(*popt))

In [ ]:
def draw_wheelhesse(wheel, option):
    fig, axs = plt.subplots(1,1, figsize=(10,5))
    plot_wheel_stationhesse(axs, wheel, 1, option)
    plot_wheel_stationhesse(axs, wheel, 2, option)
    plot_wheel_stationhesse(axs, wheel, 3, option)
    if option != "y": 
        plot_wheel_stationhesse(axs, wheel, 4, option)
    axs.set_title("Wheel {}".format(wheel))
    #hep.style.use(hep.style.ROOT)
    axs.legend()

In [ ]:
draw_wheelhesse(-2, "x")
draw_wheel(1, "x")

In [ ]:
Result.to_csv('Result.csv')

In [ ]:
Result